## Dependencies

In [40]:
# Run this if you need the SciCite model
# Requires python 3.6
# %pip install -r requirements.in -c constraints.txt 


## Analysing Data

### Imports

In [4]:
import json
import pandas as pd
import numpy as np
import os
import requests
import re
import time
import random
import matplotlib
import sys
# from imblearn.over_sampling import SMOTE

### Looking at JSON file

In [5]:
scaffold_data = []
with open("sciciteData/scaffolds/cite-worthiness-scaffold-train.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        scaffold_data.append(json.loads(line.strip()))

print(len(scaffold_data))
print(scaffold_data[0])

73484
{'text': 'The churn rate is high, with many  members leaving regularly to be replaced by new faces.', 'citing_paper_id': None, 'cited_paper_id': None, 'citing_paper_year': None, 'cited_paper_year': None, 'citing_paper_title': None, 'cited_paper_title': None, 'cited_author_ids': None, 'citing_author_ids': None, 'extended_context': None, 'section_number': None, 'section_title': None, 'intent': None, 'cite_marker_offset': None, 'sents_before': None, 'sents_after': None, 'cleaned_cite_text': 'The churn rate is high, with many  members leaving regularly to be replaced by new faces.', 'citation_id': None, 'citation_excerpt_index': None, 'section_name': None, 'is_citation': False}


### Read data

In [6]:
train_df = pd.read_json("sciciteData/train.jsonl", lines=True)
train_df

,source,citeEnd,sectionName,citeStart,string,label,label_confidence,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label2_confidence
0,explicit,175.0,Introduction,168.0,"However, how frataxin interacts with the Fe-S ...",background,1.0000,1872080baa7d30ec8fb87be9a65358cd3a7fb649,894be9b4ea46a5c422e81ef3c241072d4c73fdc0,True,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,11,NaN,NaN
1,explicit,36.0,Novel Quantitative Trait Loci for Seminal Root...,16.0,"In the study by Hickey et al. (2012), spikes w...",background,1.0000,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b,b6642e19efb8db5623b3cc4eef1c5822a6151107,True,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,2,NaN,NaN
2,explicit,228.0,Introduction,225.0,"The drug also reduces catecholamine secretion,...",background,1.0000,9cdf605beb1aa1078f235c4332b3024daa8b31dc,4e6a17fb8d7a3cada601d942e22eb5da6d01adbd,False,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,0,NaN,NaN
3,explicit,110.0,Discussion,46.0,By clustering with lowly aggressive close kin ...,background,1.0000,d9f3207db0c79a3b154f3875c9760cc6b056904b,2cc6ff899bf17666ad35893524a4d61624555ed7,False,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,3,NaN,NaN
4,explicit,239.0,Discussion,234.0,Ophthalmic symptoms are rare manifestations of...,background,1.0000,88b86556857f4374842d2af2e359576806239175,a5bb0ff1a026944d2a47a155462959af2b8505a8,False,88b86556857f4374842d2af2e359576806239175>a5bb0...,88b86556857f4374842d2af2e359576806239175>a5bb0...,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8238,explicit,50.0,,28.0,"Importantly, the results of Pascalis et al. (2...",background,0.7350,6f68ccd37718366c40ae6aeedf0b935bf560b215,60ed4bdabf92b2fbd6162dbd8979888cccca55d7,True,6f68ccd37718366c40ae6aeedf0b935bf560b215>60ed4...,6f68ccd37718366c40ae6aeedf0b935bf560b215>60ed4...,15,NaN,NaN
8239,explicit,182.0,DISCUSSION,179.0,"As suggested by Nguena et al, there is a need ...",background,0.7508,f2a1c1704f9587c94ed95bc98179dc499e933f5e,574e659da7f6c62c07bfaaacd1f31d65bd75524c,True,f2a1c1704f9587c94ed95bc98179dc499e933f5e>574e6...,f2a1c1704f9587c94ed95bc98179dc499e933f5e>574e6...,1,NaN,NaN
8240,explicit,120.0,DISCUSSION,108.0,Skeletal muscle is also a primary site of dise...,background,1.0000,18c97ea2ff60c110cc2a523e0fdf729608cbb083,fc13b9c3dfcc121013edaa12fa8ce7842aaed21a,False,18c97ea2ff60c110cc2a523e0fdf729608cbb083>fc13b...,18c97ea2ff60c110cc2a523e0fdf729608cbb083>fc13b...,8,NaN,NaN
8241,explicit,221.0,,185.0,ACTIVATION OF TRANSCRIPTION FACTORS Roles for ...,method,NaN,4ec9b89857c0b27e8a4bd3745b7358f387773527,81affdba19e38e2b17cf7b9e93792cc2028cf21d,True,4ec9b89857c0b27e8a4bd3745b7358f387773527>81aff...,4ec9b89857c0b27e8a4bd3745b7358f387773527>81aff...,0,NaN,NaN


### Read test/validation data

In [7]:
dev_df = pd.read_json("sciciteData/dev.jsonl", lines=True)
test_df = pd.read_json("sciciteData/test.jsonl", lines=True)


### Check & Clean JSON file 

In [ ]:
# Check for malformed lines
filepath = "sciciteData/train.jsonl"
valid_lines = []
with open(filepath, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        line_stripped = line.strip()
        if not line_stripped:
            continue
        try:
            record = json.loads(line_stripped)
            valid_lines.append(record)
        except json.JSONDecodeError as e:
            print(f"Line {i} is malformed! Error: {e}")

In [ ]:
# Check if all records have the required keys
cleaned_data = []
required_keys = ["source", "string", "label"]

for record in valid_lines:
    if all(key in record for key in required_keys):
        cleaned_data.append(record)
    else:
        print("Missing keys:", record)

In [ ]:
# Check for duplicates
seen = {}
duplicates = []
count = 0

for record in cleaned_data:
    record_key = (
        record.get("string", ""),
        record.get("label", ""),
        record.get("citingPaperId", ""),
        record.get("citedPaperId", "")
    )
    count = count + 1
    if record_key in seen:
        # If it's a duplicate, store both the original and the new duplicate
        print(count)
        duplicates.append({"original": seen[record_key], "duplicate": record})
    else:
        # Store the first occurrence in seen
        seen[record_key] = record

# Print duplicates for comparison
for i, pair in enumerate(duplicates):
    print(f"Duplicate {i+1}:")
    print("Original:", pair["original"])
    print("Duplicate:", pair["duplicate"])
    print("-" * 50)

# duplicates[0]
# no duplicates only 1 with same string but different citedPaperId

### Check data info

In [ ]:
train_df.info()
train_df.isnull().sum()  # check null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8243 entries, 0 to 8242
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   source             8241 non-null   object 
 1   citeEnd            8241 non-null   float64
 2   sectionName        8224 non-null   object 
 3   citeStart          8241 non-null   float64
 4   string             8243 non-null   object 
 5   label              8243 non-null   object 
 6   label_confidence   6137 non-null   float64
 7   citingPaperId      8243 non-null   object 
 8   citedPaperId       8243 non-null   object 
 9   isKeyCitation      8243 non-null   bool   
 10  id                 8243 non-null   object 
 11  unique_id          8243 non-null   object 
 12  excerpt_index      8243 non-null   int64  
 13  label2             835 non-null    object 
 14  label2_confidence  317 non-null    float64
dtypes: bool(1), float64(4), int64(1), object(9)
memory usage: 909.8+ KB


source                  2
citeEnd                 2
sectionName            19
citeStart               2
string                  0
label                   0
label_confidence     2106
citingPaperId           0
citedPaperId            0
isKeyCitation           0
id                      0
unique_id               0
excerpt_index           0
label2               7408
label2_confidence    7926
dtype: int64

In [ ]:
# Working with numeric data (but not much use in our case)
numeric_cols = ["citeStart", "citeEnd", "label_confidence"]
numeric_data = train_df[numeric_cols]
numeric_data.describe()

,citeStart,citeEnd,label_confidence
count,8241.000000,8241.000000,6137.000000
mean,139.709744,163.965781,0.876606
std,79.479383,121.478095,0.144244
min,0.000000,3.000000,0.600000
25%,87.000000,103.000000,0.744700
50%,134.000000,152.000000,1.000000
75%,187.000000,208.000000,1.000000
max,1184.000000,3305.000000,1.000000


### Data Cleaning

In [9]:
# Remove duplicates (no duplicate, only 1 with same string but different citation)
# duplicates = train_df[train_df.duplicated()]
# duplicates

# train_df.isnull().sum()
# Get the rows where column is null
# rows_with_null_col1 = train_df[train_df['source'].isnull()]
# rows_with_null_col1

# print(rows_with_nulls)

# Remove extra whitespaces
train_df["string"] = train_df["string"].str.strip()  

# Filter out irrelevant entries (all inside)
# train_df = train_df[train_df["label"].isin(["background", "method", "result"])]

### Check Null/Non-Null Column

In [ ]:
colToCheck = "label2"
rows_with_not_null_col = train_df[train_df[colToCheck].notnull()]
rows_with_not_null_col

,source,citeEnd,sectionName,citeStart,string,label,label_confidence,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label2_confidence
17,explicit,80.0,Discussion,60.0,Consistent with previous reports (al-Khodairy ...,result,0.7422,676b19ba1c40fedf4d27a3f48a766ac20b960f8e,fe2fd3262b549d1b36898127e3cc7ca1c1dd3646,True,676b19ba1c40fedf4d27a3f48a766ac20b960f8e>fe2fd...,676b19ba1c40fedf4d27a3f48a766ac20b960f8e>fe2fd...,4,supportive,0.7422
22,explicit,186.0,Discussion,175.0,These results are consistent with field observ...,result,NaN,cac9d02d7c9e7a082135858b927fa32770de68dc,None,False,cac9d02d7c9e7a082135858b927fa32770de68dc>None,cac9d02d7c9e7a082135858b927fa32770de68dc>None_0,0,not_supportive,NaN
34,explicit,133.0,DISCUSSION,83.0,More examples of contradictory results have be...,result,0.7661,39aca52b748a1e8969413295fc736ea90fd930ea,b7c9b2632ffbcef4a35a6bf0c85686d9f11c1532,True,39aca52b748a1e8969413295fc736ea90fd930ea>b7c9b...,39aca52b748a1e8969413295fc736ea90fd930ea>b7c9b...,9,not_supportive,0.5147
37,explicit,61.0,Discussion,55.0,This is in keeping with the report of Blickste...,result,0.7000,15783ea4c99fe2b855d04f3531395faff82aa95a,f2e93d20d57b3f296911c399705fa5eed006c764,True,15783ea4c99fe2b855d04f3531395faff82aa95a>f2e93...,15783ea4c99fe2b855d04f3531395faff82aa95a>f2e93...,1,supportive,0.7000
61,explicit,294.0,Discussion,274.0,"…Lemos et al., 2015) at baseline, allowed for ...",result,0.7633,9462acef826a6b495fe8fb281660ee4418bb840e,21aaceb3fc27e8cc831e25191d967daa0d8962bf,True,9462acef826a6b495fe8fb281660ee4418bb840e>21aac...,9462acef826a6b495fe8fb281660ee4418bb840e>21aac...,8,supportive,0.7633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8197,explicit,296.0,DISCUSSION,278.0,"…neurons at their long (>1,000 m) dendrites (C...",result,0.7688,ba67231cf2a97a5168c508f4eb5e85d5d647ac92,65e1507959b2afb1def77f805e5a6b615dddfa70,False,ba67231cf2a97a5168c508f4eb5e85d5d647ac92>65e15...,ba67231cf2a97a5168c508f4eb5e85d5d647ac92>65e15...,0,supportive,0.7688
8205,explicit,241.0,DISCUSSION,237.0,Our data on early differentiation of pyramidal...,result,NaN,3130c17418add1883ca4714b3407ed393617c4d6,077740cfcd5efbd90a95c432171be73fbd2369d0,False,3130c17418add1883ca4714b3407ed393617c4d6>07774...,3130c17418add1883ca4714b3407ed393617c4d6>07774...,0,not_supportive,NaN
8207,explicit,275.0,Discussion,255.0,The ¬¨¬µ-opioid receptor agonist morphine also...,result,NaN,2981321af08b96875a6b54a7ad4fbef4fbbc1b40,0baccf1d4ad8627b335d175df8eab1874ffba993,False,2981321af08b96875a6b54a7ad4fbef4fbbc1b40>0bacc...,2981321af08b96875a6b54a7ad4fbef4fbbc1b40>0bacc...,0,not_supportive,NaN
8219,explicit,218.0,Discussion,214.0,"Interestingly, E2F-1 was specifically down-reg...",result,NaN,4bf9f77ec1ba4cfffe02139cfe5f5a5e0a674ab7,decca659a694ad63933c96d97b4b64b0836c8db9,False,4bf9f77ec1ba4cfffe02139cfe5f5a5e0a674ab7>decca...,4bf9f77ec1ba4cfffe02139cfe5f5a5e0a674ab7>decca...,0,not_supportive,NaN


### Check the unique values of a certain column

In [32]:
colToCheck = "label2"
label_counts = train_df[colToCheck].value_counts()
print("Label counts:\n", label_counts)

Label counts:
 not_supportive    471
supportive        356
cant_determine      8
Name: label2, dtype: int64


### Show the specific value in the column

In [35]:
colToCheck = "label2"
valueToCheck = "cant_determine"
rows_with_value = train_df[train_df[colToCheck] == valueToCheck]
rows_with_value

,source,citeEnd,sectionName,citeStart,string,label,label_confidence,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label2_confidence
662,explicit,94.0,5.3 Experiments on the IC model,90.0,We compare SISI with the dynamic programming a...,result,0.6052,1a2114515c8aee2fb46a56fbffba0f2285570d02,72716e49b923a84db430e100feb7f6cb975f3302,True,1a2114515c8aee2fb46a56fbffba0f2285570d02>72716...,1a2114515c8aee2fb46a56fbffba0f2285570d02>72716...,3,cant_determine,0.4103
951,explicit,73.0,Discussion,69.0,comparable to that in a recent study conducted...,result,0.7312,e18dc288c2b373cdec2320479154b8645f07eadb,5ee8828669cce88d2fb768022a3e414551d8f681,False,e18dc288c2b373cdec2320479154b8645f07eadb>5ee88...,e18dc288c2b373cdec2320479154b8645f07eadb>5ee88...,8,cant_determine,0.4854
1098,explicit,257.0,4. Discussion,230.0,Synaptic heterogeneity is not the only evidenc...,result,0.7588,f1a11165ef394e3987acc4cc0056af620790b5fd,631b5d6071ec74833d779b6d5702c25a6b2be9cf,True,f1a11165ef394e3987acc4cc0056af620790b5fd>631b5...,f1a11165ef394e3987acc4cc0056af620790b5fd>631b5...,5,cant_determine,0.5231
2234,explicit,41.0,Statistical analysis,18.0,Data derived from Rosengaus et al. (1998) was ...,result,0.7322,f9e9bef98a81a43ce7b700822e03423273186029,7b2b2256858a4d7d4a273db9dd3a643b2fadc055,False,f9e9bef98a81a43ce7b700822e03423273186029>7b2b2...,f9e9bef98a81a43ce7b700822e03423273186029>7b2b2...,4,cant_determine,0.4863
4234,explicit,85.0,"John M. Howell,1 Thom A. Mayer,1 Dan Hanfling,...",79.0,We retrospectively compared 2 methods that hav...,result,0.6096,a42da1ce9099efd398b549925dc770bf25e07b21,d667b306dcd2badf6b37970c3cb63b4b8512d9bf,True,a42da1ce9099efd398b549925dc770bf25e07b21>d667b...,a42da1ce9099efd398b549925dc770bf25e07b21>d667b...,4,cant_determine,0.4050
4811,explicit,57.0,3.3. Enzymatic hydrolysis,40.0,This is in comparison to previous work (Pan et...,result,0.6834,fe677d2f86d623b3dfc92f060bd8d0d2a0ea413d,548992f59f682a83a671e7f288f7aa574580d4a5,True,fe677d2f86d623b3dfc92f060bd8d0d2a0ea413d>54899...,fe677d2f86d623b3dfc92f060bd8d0d2a0ea413d>54899...,0,cant_determine,0.3436
4849,explicit,238.0,Discussion,220.0,"…(2005), several differences between CPP proto...",result,0.7180,07895f7b08a16c607fc6e8621a1256784c7461c4,488e1b26dcc9274c9bf4f8ee7dd5d96109565214,True,07895f7b08a16c607fc6e8621a1256784c7461c4>488e1...,07895f7b08a16c607fc6e8621a1256784c7461c4>488e1...,0,cant_determine,0.2498
5987,explicit,105.0,4 Experimental results,101.0,In order to evaluate the effectiveness and eff...,result,0.6068,979dd2651ff2f04c52095a60978ed3bc87610062,777896b0f4cb6d2823e15a9265b890c0db9b6de5,True,979dd2651ff2f04c52095a60978ed3bc87610062>77789...,979dd2651ff2f04c52095a60978ed3bc87610062>77789...,5,cant_determine,0.6068


In [ ]:
# Check the specific row in that column
rows_with_value.iloc[7]["string"] 

'In order to evaluate the effectiveness and efficiency of OLCS, we compare its performance with CLPSO [23], CMA-ES [24], and GL-25 [25].'

In [ ]:
train_df.iloc[6439]

## Pre-processing

### Using Semantic Scholar (datasets)

In [ ]:
# Get list of available releases
releases_url = "https://api.semanticscholar.org/datasets/v1/release"
response = requests.get(releases_url)
releases = response.json()
print(releases[-1])

# Choose a release and get dataset links
datasets_url = "https://api.semanticscholar.org/datasets/v1/release/latest"
response = requests.get(datasets_url)
datasets = response.json()
print(datasets['release_id'])

datasets


In [ ]:
# Get the latest dataset release
release = requests.get('https://api.semanticscholar.org/datasets/v1/release/latest/dataset/abstracts').json()
print(json.dumps(release, indent=2))

### Get the specific paper

In [39]:
paper_id = "d9f3207db0c79a3b154f3875c9760cc6b056904b"
fields = "title,year,citationCount,referenceCount,fieldsOfStudy"
url = f"https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields={fields}"
response = requests.get(url)
datasets = response.json()

datasets

{'message': 'Too Many Requests. Please wait and try again or apply for a key for higher rate limits. https://www.semanticscholar.org/product/api#api-key-form',
 'code': '429'}

## Augmenting with PubMed 20k Data

In [26]:
pm_train_df = pd.read_csv("pubmed_20k/train.csv")

In [11]:
pm_train_df


,abstract_id,line_id,abstract_text,line_number,total_lines,target
0,24293578,24293578_0_12,To investigate the efficacy of 6 weeks of dail...,0,12,OBJECTIVE
1,24293578,24293578_1_12,A total of 125 patients with primary knee OA w...,1,12,METHODS
2,24293578,24293578_2_12,Outcome measures included pain reduction and i...,2,12,METHODS
3,24293578,24293578_3_12,Pain was assessed using the visual analog pain...,3,12,METHODS
4,24293578,24293578_4_12,Secondary outcome measures included the Wester...,4,12,METHODS
...,...,...,...,...,...,...
180035,26227186,26227186_7_12,For the absolute change in percent atheroma vo...,7,12,RESULTS
180036,26227186,26227186_8_12,"For PAV , a significantly greater percentage o...",8,12,RESULTS
180037,26227186,26227186_9_12,Both strategies had acceptable side effect pro...,9,12,RESULTS
180038,26227186,26227186_10_12,"Compared with standard statin monotherapy , th...",10,12,CONCLUSIONS


In [8]:
pm_train_df.iloc[1]['abstract_text']

'A total of 125 patients with primary knee OA were randomized 1:1 ; 63 received 7.5 mg/day of prednisolone and 62 received placebo for 6 weeks .'

### Clean PubMed Data

In [27]:
# Remove extra whitespaces
pm_train_df["abstract_text"] = pm_train_df["abstract_text"].str.strip()

In [13]:
# See info
pm_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180040 entries, 0 to 180039
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   abstract_id    180040 non-null  int64 
 1   line_id        180040 non-null  object
 2   abstract_text  180040 non-null  object
 3   line_number    180040 non-null  int64 
 4   total_lines    180040 non-null  int64 
 5   target         180040 non-null  object
dtypes: int64(3), object(3)
memory usage: 8.2+ MB


### Check for unique labels and its value

In [33]:
colToCheck = "target"
label_counts = pm_train_df[colToCheck].value_counts()
print("Label counts:\n", label_counts)

Label counts:
 METHODS        59353
RESULTS        57953
CONCLUSIONS    27168
BACKGROUND     21727
OBJECTIVE      13839
Name: target, dtype: int64


In [12]:
train_df.head(2)

,source,citeEnd,sectionName,citeStart,string,label,label_confidence,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label2_confidence
0,explicit,175.0,Introduction,168.0,"However, how frataxin interacts with the Fe-S ...",background,1.0,1872080baa7d30ec8fb87be9a65358cd3a7fb649,894be9b4ea46a5c422e81ef3c241072d4c73fdc0,True,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,11,NaN,NaN
1,explicit,36.0,Novel Quantitative Trait Loci for Seminal Root...,16.0,"In the study by Hickey et al. (2012), spikes w...",background,1.0,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b,b6642e19efb8db5623b3cc4eef1c5822a6151107,True,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,2,NaN,NaN


### Difference between id and unique id

In [125]:
idxToCheck = 0

train_df.iloc[idxToCheck]['id']

'1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be9b4ea46a5c422e81ef3c241072d4c73fdc0'

In [ ]:
train_df.iloc[idxToCheck]['unique_id']

# unique id has an extra excerpt id at the back

'1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be9b4ea46a5c422e81ef3c241072d4c73fdc0_11'

### Sample PubMed Data Function

In [29]:
def sample_pubmed(pubmed_df, scicite_count, ratio=0.30):    
    # Calculate the number of PubMed lines needed
    T = scicite_count / (1 - ratio)
    pubmed_needed = int(round(ratio * T))

    # Get unique abstract IDs
    unique_abstracts = pubmed_df['abstract_id'].unique()

    # Shuffle abstracts
    np.random.seed(42)
    np.random.shuffle(unique_abstracts)

    selected_abstracts = []
    total_rows = 0

    for abstract_id in unique_abstracts:
        abstract_rows = pubmed_df[pubmed_df['abstract_id'] == abstract_id]
        selected_abstracts.append(abstract_rows)
        total_rows += len(abstract_rows)
        
        if total_rows >= pubmed_needed:
            break

    # Combine abstracts list into a single DataFrame
    sampled_df = pd.concat(selected_abstracts, ignore_index=True)

    return sampled_df

### Check for unique values

In [16]:
# unique id
len(pm_train_df['abstract_id'].unique())
pm_train_df['abstract_id'].value_counts()

# id = 24761898
# pm_train_df[pm_train_df['abstract_id'] == id]
# pm_train_df

25105787    31
24607076    29
25130995    29
25518308    28
24471250    27
            ..
25493493     4
24899076     4
25390467     4
24761898     4
25314069     4
Name: abstract_id, Length: 15000, dtype: int64

### Convert PubMed to Scicite Format Function (output to jsonl)

In [30]:
# Change to match SciCite dataset format
def map_pubmed_to_scicite_label(pubmed_label):
    pubmed_label = pubmed_label.upper()
    if pubmed_label == "BACKGROUND": 
        return "background"
    elif pubmed_label == "METHODS":
        return "method"
    elif pubmed_label == "RESULTS":
        return "result"
    else:
        return None

def convert_pubmed_to_scicite(pubmed_df, scicite_count, out_jsonl, ratio):
    # Convert the "target" labels to SciCite format
    pubmed_df["scicite_label"] = pubmed_df["target"].map(map_pubmed_to_scicite_label)
    filtered_pubmed_df = pubmed_df.dropna(subset=["scicite_label"])

    # Sample from the filtered dataset
    sampled_pubmed_df = sample_pubmed(filtered_pubmed_df, scicite_count, ratio)

    scicite_list = []
    
    for _, row in sampled_pubmed_df.iterrows():
        # Read from PubMed Data
        abs_id = row["abstract_id"]
        splited_line_id = row["line_id"].split("_")
        line_num = str(int(splited_line_id[-2]) + 1) 
        total_lines = splited_line_id[-1]
        line_id = f"{line_num}_{total_lines}"

        text = row["abstract_text"]
        label = row["scicite_label"]

        # Map label
        # scicite_label = map_pubmed_to_scicite_label(original_label)

        # drop the irrelevant ones
        # if scicite_label is None:
            # continue
        
        
        # SciCite format
        citing_id = f"PMID_{abs_id}"
        cited_id = f"PMID_{abs_id}"
        line_str = text.strip()
        unique_id = f"{citing_id}>{cited_id}_line_{line_id}"

        scicite_obj = {
            "source": "explicit",
            "citeStart": 0,
            "citeEnd": 0,
            "sectionName": None,
            "string": line_str,
            "label": label,
            "label_confidence": None,
            "citingPaperId": citing_id,
            "citedPaperId": cited_id,
            "isKeyCitation": False,
            "id": f"{citing_id}>{cited_id}",
            "unique_id": unique_id,
            "excerpt_index": int(line_num),
            "label2": None,  
            "label2_confidence": None
            
        }
        
        scicite_list.append(scicite_obj)

    # Dump them as JSONL
    with open(out_jsonl, "w", encoding="utf-8") as fout:
        for item in scicite_list:
            line = json.dumps(item, ensure_ascii=False)
            fout.write(line + "\n")




### Run Conversion

In [31]:
scicite_count = len(train_df)

if not os.path.exists("AugmentedData"):
    os.makedirs("AugmentedData")

output_file = "AugmentedData/pubmed_augmented.jsonl"
ratio = 0.30
convert_pubmed_to_scicite(pm_train_df, len(train_df), output_file, ratio)

### Read Augmented PubMed Data

In [32]:
pubmed_augmented_df = pd.read_json("AugmentedData/pubmed_augmented.jsonl", lines=True)
pubmed_augmented_df

,source,citeStart,citeEnd,sectionName,string,label,label_confidence,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label2_confidence
0,explicit,0,0,NaN,The PREDIMED study is a randomized nutritional...,method,NaN,PMID_25250626,PMID_25250626,False,PMID_25250626>PMID_25250626,PMID_25250626>PMID_25250626_line_4_14,4,NaN,NaN
1,explicit,0,0,NaN,Dietary information was collected at baseline ...,method,NaN,PMID_25250626,PMID_25250626,False,PMID_25250626>PMID_25250626,PMID_25250626>PMID_25250626_line_5_14,5,NaN,NaN
2,explicit,0,0,NaN,We assigned GI values of each item by a 5-step...,method,NaN,PMID_25250626,PMID_25250626,False,PMID_25250626>PMID_25250626,PMID_25250626>PMID_25250626_line_6_14,6,NaN,NaN
3,explicit,0,0,NaN,Deaths were ascertained through contact with f...,method,NaN,PMID_25250626,PMID_25250626,False,PMID_25250626>PMID_25250626,PMID_25250626>PMID_25250626_line_7_14,7,NaN,NaN
4,explicit,0,0,NaN,Cox regression models were used to estimate mu...,method,NaN,PMID_25250626,PMID_25250626,False,PMID_25250626>PMID_25250626,PMID_25250626>PMID_25250626_line_8_14,8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3533,explicit,0,0,NaN,Rescue analgesia was provided with an intraven...,method,NaN,PMID_25768238,PMID_25768238,False,PMID_25768238>PMID_25768238,PMID_25768238>PMID_25768238_line_8_14,8,NaN,NaN
3534,explicit,0,0,NaN,"Compared with Group I , Group III patients had...",result,NaN,PMID_25768238,PMID_25768238,False,PMID_25768238>PMID_25768238,PMID_25768238>PMID_25768238_line_9_14,9,NaN,NaN
3535,explicit,0,0,NaN,Group III patients also recorded significantly...,result,NaN,PMID_25768238,PMID_25768238,False,PMID_25768238>PMID_25768238,PMID_25768238>PMID_25768238_line_10_14,10,NaN,NaN
3536,explicit,0,0,NaN,There was no significant difference in shoulde...,result,NaN,PMID_25768238,PMID_25768238,False,PMID_25768238>PMID_25768238,PMID_25768238>PMID_25768238_line_11_14,11,NaN,NaN


### Merge SciCite and PubMed Data

In [33]:
# Merge the two datasets
augmented_df = pd.concat([train_df, pubmed_augmented_df], ignore_index=True)


In [34]:
augmented_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11781 entries, 0 to 11780
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   source             11779 non-null  object 
 1   citeEnd            11779 non-null  float64
 2   sectionName        8224 non-null   object 
 3   citeStart          11779 non-null  float64
 4   string             11781 non-null  object 
 5   label              11781 non-null  object 
 6   label_confidence   6137 non-null   float64
 7   citingPaperId      11781 non-null  object 
 8   citedPaperId       11781 non-null  object 
 9   isKeyCitation      11781 non-null  bool   
 10  id                 11781 non-null  object 
 11  unique_id          11781 non-null  object 
 12  excerpt_index      11781 non-null  int64  
 13  label2             835 non-null    object 
 14  label2_confidence  317 non-null    float64
dtypes: bool(1), float64(4), int64(1), object(9)
memory usage: 1.3+ MB


## Run Model

### Imports

In [ ]:
# %pip uninstall allennlp -y
# %pip uninstall allennlp-models -y
# %pip install allennlp==0.8
# !pip freeze
# %pip install jsonnet-binary
# Not sure if the 2 below helps, but I did it (can try without it)
# %pip uninstall greenlet gevent
# %pip install --no-cache-dir greenlet==0.4.17 gevent==1.3.7

from allennlp.models import *
from allennlp.models.archival import load_archive
from allennlp.predictors import Predictor

# Change to the path of your scicite-master folder
# /path/to/scicite-master
# directory structure should be like this:
# scicite-master
# - scicite
#   -- models
#       --- scaffolds
sys.path.append(r"C:\Users\PC\OneDrive - National University of Singapore\Year 3.2\CS4248\Project\scicite-master")
from scicite.models.scaffold_bilstm_attention_classifier import ScaffoldBilstmAttentionClassifier

### Load SciCite Pretrained Model

In [28]:
# might need to install these packages
# %pip install jsonlines
# %pip install Levenshtein
# !python -m spacy download en_core_web_sm

predictor = Predictor.from_path("scicite.tar.gz", predictor_name="predictor_aclarc")
print("Model loaded!")

04/01/2025 04:44:57 - INFO - allennlp.models.archival -   loading archive file scicite.tar.gz
04/01/2025 04:44:57 - INFO - allennlp.models.archival -   extracting archive file scicite.tar.gz to temp dir C:\Users\PC\AppData\Local\Temp\tmpsmlj2so8
04/01/2025 04:45:02 - INFO - allennlp.common.params -   type = default
04/01/2025 04:45:02 - INFO - allennlp.data.vocabulary -   Loading token dictionary from C:\Users\PC\AppData\Local\Temp\tmpsmlj2so8\vocabulary.
04/01/2025 04:45:02 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.models.model.Model'> from params {'citation_text_encoder': {'bidirectional': True, 'dropout': 0.3, 'hidden_size': 100, 'input_size': 1124, 'num_layers': 2, 'type': 'gru'}, 'classifier_feedforward': {'activations': ['linear', 'linear'], 'dropout': [0, 0], 'hidden_dims': [20, 3], 'input_dim': 200, 'num_layers': 2}, 'classifier_feedforward_3': {'activations': ['linear', 'linear'], 'dropout': [0, 0], 'hidden_dims': [20, 2], 'input_dim': 200, 

pred mode: False


04/01/2025 04:45:06 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.data.dataset_readers.dataset_reader.DatasetReader'> from params {'multilabel': False, 'type': 'scicite_datasetreader', 'use_sparse_lexicon_features': False, 'with_elmo': 'true'} and extras {}
04/01/2025 04:45:06 - INFO - allennlp.common.params -   dataset_reader.type = scicite_datasetreader
04/01/2025 04:45:06 - INFO - allennlp.common.params -   dataset_reader.lazy = False
04/01/2025 04:45:06 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.data.tokenizers.tokenizer.Tokenizer'> from params {} and extras {}
04/01/2025 04:45:06 - INFO - allennlp.common.params -   dataset_reader.tokenizer.type = word
04/01/2025 04:45:06 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.data.tokenizers.word_tokenizer.WordTokenizer'> from params {} and extras {}
04/01/2025 04:45:06 - INFO - allennlp.common.params -   dataset_reader.tokenizer.start_tokens 

Model loaded!


In [ ]:
pubmed_dict = pubmed_augmented_df[["string", "citingPaperId", "citedPaperId", "sectionName", "label", "excerpt_index"]].to_dict(orient="records")

pubmed_dict[0]
# Take about 20 minutes to run
# predictions = [predictor.predict_json(item) for item in pubmed_dict]


In [ ]:
print(predictions[0].keys())

max(predictions[0]["probabilities"])

In [ ]:
pubmed_augmented_df["predicted_label"] = [pred["prediction"] for pred in predictions]
# pubmed_augmented_df["predicted_label_confidence"] = [pred["probabilities"] for pred in predictions]

# compare the predicted label with the original label
matches = (pubmed_augmented_df["predicted_label"] == pubmed_augmented_df["label"]).sum()
mismatches = len(pubmed_augmented_df) - matches

# check how many are same
print(f"Matches: {matches}, Mismatches: {mismatches}")

mismatched_df = pubmed_augmented_df[pubmed_augmented_df["predicted_label"] != pubmed_augmented_df["label"]]
mismatched_df[["string", "label", "predicted_label"]].head(10)



In [62]:
idxToCheck = 7

mismatched_df.iloc[idxToCheck]['string']


'Intervention effects were found for a ) weekly alcohol consumption at 1 , 2 , 3 , 4 , and 7 weeks follow-up , b ) frequency of binge drinking at 1 , 2 , 7 , and 12 weeks follow-up , and c ) heavy drinking status at 1 , 2 , 7 , and 16 weeks follow-up .'

### Save the Final Augmented Data

In [39]:
# Extract augmented dataframe to jsonl
augmented_df.to_json("AugmentedData/augmented_data.jsonl", orient="records", lines=True, force_ascii=False)